
# **数据挖掘——Home Credit Default Risk**

Authors：李林（3120220938）、杨洋（3220211141）、敬甲男（3220221052）、李翰杰（3120220936）

github地址：https://github.com/leealim/kaggle-Home-Credit-Default-Risk

---

## 数据预处理——缺失值处理

共八张表，逐个进行处理：
- application_{train|test}.csv:客户申请表
- bureau.csv/bureau_balance.csv:客户历史借款记录
- POS_CASH_balance.csv:客户POS和现金贷款历史
- credit_card_balance.csv:客户信用卡的snapshot历史
- previous_application.csv:客户历史申请记录
- installments_payments.csv:客户信用卡还款记录

---


In [52]:
# 引入本部分所需要的包，并定义需要的值和函数

import pandas as pd
import os

source_dir="..\\data\\home-credit-default-risk"
app_tr_path = source_dir+"\\application_train.csv"
app_te_path = source_dir+"\\application_test.csv"
bur_path = source_dir+"\\bureau.csv"
bur_bal_path = source_dir+"\\bureau_balance.csv"
pos_path = source_dir+"\\POS_CASH_balance.csv"
cre_path = source_dir+"\\credit_card_balance.csv"
pre_path = source_dir+"\\previous_application.csv"
ins_path = source_dir+"\\installments_payments.csv"

hom_path = "..\\data\\home-credit-default-risk\\HomeCredit_columns_description.csv"  # 列描述表
hom = pd.read_csv(hom_path)

result_dir="..\\data\\miss_value_handling"
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

def missing_values_table(df, table_name):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table = mis_val_table.rename(
        columns={0: 'Missing Values',
                 1: '% of Total Values'})
    mis_val_table = mis_val_table.sort_values(
        '% of Total Values', ascending=False).round(1)
    miss_num = (mis_val_table["Missing Values"] != 0).sum()
    print("Total " + str(miss_num) + " columns missing values")
    mis_val_table = mis_val_table.drop(index=mis_val_table[miss_num:].index)
    mis_val_table = mis_val_table.merge(
        hom, how="left", left_index=True, right_on='Row')
    mis_val_table = mis_val_table.drop(columns=['Unnamed: 0'])
    mis_val_table = mis_val_table.drop(
        index=mis_val_table.loc[mis_val_table["Table"] != table_name].index)
    mis_val_table = mis_val_table.reindex(
        columns=["Row", "Description", "Special", "Missing Values", "% of Total Values"])
    mis_val_table = mis_val_table.reset_index(drop=True)
    mis_val_table = mis_val_table.merge(
        df.describe().T, how="left", left_on="Row", right_index=True)
    return mis_val_table



### 1. **application_{train|test}.csv**

In [53]:
# 查看训练数据的基本数据特征

app_tr = pd.read_csv(app_tr_path)
app_tr.describe()


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,...,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,...,0.008130,0.000595,0.000507,0.000335,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,102790.175348,0.272419,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.013831,4363.988632,141275.766519,...,0.089798,0.024387,0.022518,0.018299,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189145.500000,0.000000,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.010006,-19682.000000,-2760.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278202.000000,0.000000,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15750.000000,-1213.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367142.500000,0.000000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12413.000000,-289.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.000000,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,365243.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


In [54]:
# 缺失值分析

t = missing_values_table(app_tr, "application_{train|test}.csv")
pd.set_option("display.max_rows", 20)
pd.set_option('max_colwidth',40)
t


Total 67 columns missing values


,Row,Description,Special,Missing Values,% of Total Values,count,mean,std,min,25%,50%,75%,max
0,COMMONAREA_MEDI,Normalized information about buildin...,normalized,214865,69.9,92646.0,0.044595,0.076144,0.000000e+00,0.007900,0.020800,0.051300,1.000
1,COMMONAREA_AVG,Normalized information about buildin...,normalized,214865,69.9,92646.0,0.044621,0.076036,0.000000e+00,0.007800,0.021100,0.051500,1.000
2,COMMONAREA_MODE,Normalized information about buildin...,normalized,214865,69.9,92646.0,0.042553,0.074445,0.000000e+00,0.007200,0.019000,0.049000,1.000
3,NONLIVINGAPARTMENTS_MODE,Normalized information about buildin...,normalized,213514,69.4,93997.0,0.008076,0.046276,0.000000e+00,0.000000,0.000000,0.003900,1.000
4,NONLIVINGAPARTMENTS_AVG,Normalized information about buildin...,normalized,213514,69.4,93997.0,0.008809,0.047732,0.000000e+00,0.000000,0.000000,0.003900,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,EXT_SOURCE_2,Normalized score from external data ...,normalized,660,0.2,306851.0,0.514393,0.191060,8.173617e-08,0.392457,0.565961,0.663617,0.855
63,AMT_GOODS_PRICE,For consumer loans it is the price o...,NaN,278,0.1,307233.0,538396.207429,369446.460540,4.050000e+04,238500.000000,450000.000000,679500.000000,4050000.000
64,AMT_ANNUITY,Loan annuity,NaN,12,0.0,307499.0,27108.573909,14493.737315,1.615500e+03,16524.000000,24903.000000,34596.000000,258025.500
65,CNT_FAM_MEMBERS,How many family members does client ...,NaN,2,0.0,307509.0,2.152665,0.910682,1.000000e+00,2.000000,2.000000,3.000000,20.000


In [55]:
#获取较小的缺失值列信息

t_small=t.loc[t["% of Total Values"]<3]
t_small


,Row,Description,Special,Missing Values,% of Total Values,count,mean,std,min,25%,50%,75%,max
57,NAME_TYPE_SUITE,Who was accompanying client when he ...,NaN,1292,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,OBS_30_CNT_SOCIAL_CIRCLE,How many observation of client's soc...,NaN,1021,0.3,306490.0,1.422245,2.400989,0.000000e+00,0.000000,0.000000,2.000000,348.000
59,DEF_30_CNT_SOCIAL_CIRCLE,How many observation of client's soc...,NaN,1021,0.3,306490.0,0.143421,0.446698,0.000000e+00,0.000000,0.000000,0.000000,34.000
60,OBS_60_CNT_SOCIAL_CIRCLE,How many observation of client's soc...,NaN,1021,0.3,306490.0,1.405292,2.379803,0.000000e+00,0.000000,0.000000,2.000000,344.000
61,DEF_60_CNT_SOCIAL_CIRCLE,How many observation of client's soc...,NaN,1021,0.3,306490.0,0.100049,0.362291,0.000000e+00,0.000000,0.000000,0.000000,24.000
62,EXT_SOURCE_2,Normalized score from external data ...,normalized,660,0.2,306851.0,0.514393,0.191060,8.173617e-08,0.392457,0.565961,0.663617,0.855
63,AMT_GOODS_PRICE,For consumer loans it is the price o...,NaN,278,0.1,307233.0,538396.207429,369446.460540,4.050000e+04,238500.000000,450000.000000,679500.000000,4050000.000
64,AMT_ANNUITY,Loan annuity,NaN,12,0.0,307499.0,27108.573909,14493.737315,1.615500e+03,16524.000000,24903.000000,34596.000000,258025.500
65,CNT_FAM_MEMBERS,How many family members does client ...,NaN,2,0.0,307509.0,2.152665,0.910682,1.000000e+00,2.000000,2.000000,3.000000,20.000
66,DAYS_LAST_PHONE_CHANGE,How many days before application did...,NaN,1,0.0,307510.0,-962.858788,826.808487,-4.292000e+03,-1570.000000,-757.000000,-274.000000,0.000


In [56]:
#获取较大的缺失值列信息

t_large=t.loc[t["% of Total Values"]>3]
t_large

,Row,Description,Special,Missing Values,% of Total Values,count,mean,std,min,25%,50%,75%,max
0,COMMONAREA_MEDI,Normalized information about buildin...,normalized,214865,69.9,92646.0,0.044595,0.076144,0.0,0.0079,0.0208,0.0513,1.0
1,COMMONAREA_AVG,Normalized information about buildin...,normalized,214865,69.9,92646.0,0.044621,0.076036,0.0,0.0078,0.0211,0.0515,1.0
2,COMMONAREA_MODE,Normalized information about buildin...,normalized,214865,69.9,92646.0,0.042553,0.074445,0.0,0.0072,0.0190,0.0490,1.0
3,NONLIVINGAPARTMENTS_MODE,Normalized information about buildin...,normalized,213514,69.4,93997.0,0.008076,0.046276,0.0,0.0000,0.0000,0.0039,1.0
4,NONLIVINGAPARTMENTS_AVG,Normalized information about buildin...,normalized,213514,69.4,93997.0,0.008809,0.047732,0.0,0.0000,0.0000,0.0039,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,AMT_REQ_CREDIT_BUREAU_DAY,Number of enquiries to Credit Bureau...,NaN,41519,13.5,265992.0,0.007000,0.110757,0.0,0.0000,0.0000,0.0000,9.0
53,AMT_REQ_CREDIT_BUREAU_WEEK,Number of enquiries to Credit Bureau...,NaN,41519,13.5,265992.0,0.034362,0.204685,0.0,0.0000,0.0000,0.0000,8.0
54,AMT_REQ_CREDIT_BUREAU_MON,Number of enquiries to Credit Bureau...,NaN,41519,13.5,265992.0,0.267395,0.916002,0.0,0.0000,0.0000,0.0000,27.0
55,AMT_REQ_CREDIT_BUREAU_QRT,Number of enquiries to Credit Bureau...,NaN,41519,13.5,265992.0,0.265474,0.794056,0.0,0.0000,0.0000,0.0000,261.0


可以发现，残缺值数量差距很大，对于小于百分之一的残缺值，我们采取删去对应行的措施。


In [57]:
#删去部分行

app_tr.dropna(subset=t_small["Row"],
          axis=0, # axis=0表示删除行；
          how='any', # how=any表示若列name、age中，任意一个出现空值，就删掉该行
          inplace=True # inplace=True表示在原df上进行修改；
          )
app_tr = app_tr.reset_index(drop=True)
app_tr

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304526,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
304527,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
304528,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
304529,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


对于残缺值比较大的行，我们逐一进行研究处理。首先，对每个特征融入统计数据。

In [58]:
t = missing_values_table(app_tr, "application_{train|test}.csv")
pd.set_option("display.max_rows", 400)
pd.set_option('max_colwidth',400)
t




Total 57 columns missing values


,Row,Description,Special,Missing Values,% of Total Values,count,mean,std,min,25%,50%,75%,max
0,COMMONAREA_MEDI,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,212870,69.9,91661.0,0.044544,0.076043,0.000000,0.007900,0.020800,0.051300,1.000000
1,COMMONAREA_AVG,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,212870,69.9,91661.0,0.044564,0.075932,0.000000,0.007800,0.021100,0.051400,1.000000
2,COMMONAREA_MODE,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,212870,69.9,91661.0,0.042510,0.074343,0.000000,0.007200,0.019000,0.048900,1.000000
3,NONLIVINGAPARTMENTS_MODE,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,211544,69.5,92987.0,0.008061,0.046265,0.000000,0.000000,0.000000,0.003900,1.000000
4,NONLIVINGAPARTMENTS_AVG,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,211544,69.5,92987.0,0.008795,0.047732,0.000000,0.000000,0.000000,0.003900,1.000000
5,NONLIVINGAPARTMENTS_MEDI,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,211544,69.5,92987.0,0.008637,0.047412,0.000000,0.000000,0.000000,0.003900,1.000000
6,FONDKAPREMONT_MODE,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,208352,68.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,LIVINGAPARTMENTS_MODE,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,208259,68.4,96272.0,0.105537,0.097673,0.000000,0.054200,0.076200,0.131300,1.000000
8,LIVINGAPARTMENTS_AVG,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,208259,68.4,96272.0,0.100662,0.092368,0.000000,0.050400,0.075600,0.121000,1.000000
9,LIVINGAPARTMENTS_MEDI,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor",normalized,208259,68.4,96272.0,0.101845,0.093431,0.000000,0.051300,0.076100,0.123100,

可以发现还存在一些存在大量残缺值的分类数据。对于这些数据，在转化为数值数据时，多转化一个类别。
另外，这里面有着大量缺失的房产数据，可以简化这些特征的拥有总和为一个特征，表明拥有房产的可信度。



In [59]:
#填补分类数据缺失值

temp=t.drop(columns=["Special"]).isnull().T.any()
temp.loc[temp==True].index
rows=t.loc[temp.loc[temp==True].index].Row

for col in rows: 
    app_tr[str(col)] = app_tr[str(col)].fillna(value="MyNull")
    
app_tr[str(rows.iloc[0])]

0         reg oper account
1         reg oper account
2                   MyNull
3                   MyNull
4                   MyNull
                ...       
304526    reg oper account
304527    reg oper account
304528    reg oper account
304529              MyNull
304530              MyNull
Name: FONDKAPREMONT_MODE, Length: 304531, dtype: object

In [60]:
t=missing_values_table(app_tr,"application_{train|test}.csv")
pd.set_option("display.max_rows", 20)
pd.set_option('max_colwidth',40)
t

Total 52 columns missing values


,Row,Description,Special,Missing Values,% of Total Values,count,mean,std,min,25%,50%,75%,max
0,COMMONAREA_AVG,Normalized information about buildin...,normalized,212870,69.9,91661.0,0.044564,0.075932,0.0,0.0078,0.0211,0.0514,1.0
1,COMMONAREA_MEDI,Normalized information about buildin...,normalized,212870,69.9,91661.0,0.044544,0.076043,0.0,0.0079,0.0208,0.0513,1.0
2,COMMONAREA_MODE,Normalized information about buildin...,normalized,212870,69.9,91661.0,0.042510,0.074343,0.0,0.0072,0.0190,0.0489,1.0
3,NONLIVINGAPARTMENTS_MODE,Normalized information about buildin...,normalized,211544,69.5,92987.0,0.008061,0.046265,0.0,0.0000,0.0000,0.0039,1.0
4,NONLIVINGAPARTMENTS_AVG,Normalized information about buildin...,normalized,211544,69.5,92987.0,0.008795,0.047732,0.0,0.0000,0.0000,0.0039,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,AMT_REQ_CREDIT_BUREAU_DAY,Number of enquiries to Credit Bureau...,NaN,41108,13.5,263423.0,0.006981,0.110358,0.0,0.0000,0.0000,0.0000,9.0
48,AMT_REQ_CREDIT_BUREAU_WEEK,Number of enquiries to Credit Bureau...,NaN,41108,13.5,263423.0,0.034484,0.204615,0.0,0.0000,0.0000,0.0000,8.0
49,AMT_REQ_CREDIT_BUREAU_MON,Number of enquiries to Credit Bureau...,NaN,41108,13.5,263423.0,0.267782,0.915330,0.0,0.0000,0.0000,0.0000,27.0
50,AMT_REQ_CREDIT_BUREAU_QRT,Number of enquiries to Credit Bureau...,NaN,41108,13.5,263423.0,0.266127,0.795735,0.0,0.0000,0.0000,0.0000,261.0


In [61]:
#重新理解房产数据,暂时存储删除列和test表合并删除，并将删除列的空值填补任意值。
print(app_tr.shape)
t_house=t.loc[t["Description"]=="Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor"]
temp=app_tr.loc[:,t_house["Row"].tolist()].isnull().sum(axis=1)
app_tr["MY_HOUSE_OWN"]=temp
app_tr_drop_list=t_house["Row"].tolist()
for col in app_tr_drop_list: 
    app_tr[col] = app_tr[col].fillna(value=0)
app_tr

(304531, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,MY_HOUSE_OWN
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,43
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304526,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
304527,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
304528,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0,3
304529,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,27


In [62]:
t=missing_values_table(app_tr,"application_{train|test}.csv")
t

Total 9 columns missing values


,Row,Description,Special,Missing Values,% of Total Values,count,mean,std,min,25%,50%,75%,max
0,OWN_CAR_AGE,Age of client's car,NaN,200912,66.0,103619.0,12.070682,11.935821,0.000000,5.000000,9.000000,15.000000,91.000000
1,EXT_SOURCE_1,Normalized score from external data ...,normalized,171652,56.4,132879.0,0.501986,0.211049,0.014568,0.333967,0.505819,0.674901,0.962693
2,EXT_SOURCE_3,Normalized score from external data ...,normalized,60251,19.8,244280.0,0.510764,0.194843,0.000527,0.370650,0.535276,0.669057,0.896010
3,AMT_REQ_CREDIT_BUREAU_YEAR,Number of enquiries to Credit Bureau...,NaN,41108,13.5,263423.0,1.905904,1.869645,0.000000,0.000000,1.000000,3.000000,25.000000
4,AMT_REQ_CREDIT_BUREAU_QRT,Number of enquiries to Credit Bureau...,NaN,41108,13.5,263423.0,0.266127,0.795735,0.000000,0.000000,0.000000,0.000000,261.000000
5,AMT_REQ_CREDIT_BUREAU_MON,Number of enquiries to Credit Bureau...,NaN,41108,13.5,263423.0,0.267782,0.915330,0.000000,0.000000,0.000000,0.000000,27.000000
6,AMT_REQ_CREDIT_BUREAU_WEEK,Number of enquiries to Credit Bureau...,NaN,41108,13.5,263423.0,0.034484,0.204615,0.000000,0.000000,0.000000,0.000000,8.000000
7,AMT_REQ_CREDIT_BUREAU_DAY,Number of enquiries to Credit Bureau...,NaN,41108,13.5,263423.0,0.006981,0.110358,0.000000,0.000000,0.000000,0.000000,9.000000
8,AMT_REQ_CREDIT_BUREAU_HOUR,Number of enquiries to Credit Bureau...,NaN,41108,13.5,263423.0,0.006385,0.083786,0.000000,0.000000,0.000000,0.000000,4.000000


目前，还有9列缺失值，可以发现OWN_CAR_AGE是车辆拥有年限数据，缺失值可以置零   
可以看到EXT_SOURCE_1和EXT_SOURCE_3比较平滑，直接用平均值代替  
AMT_REQ_CREDIT_BUREAU的一列数据用0填补，缺失值可能说明这些客户并未有对应的enquiries

In [63]:
#处理OWN_CAR_AGE

app_tr["OWN_CAR_AGE"] = app_tr["OWN_CAR_AGE"].fillna(value=0)
app_tr["OWN_CAR_AGE"]


0          0.0
1          0.0
2         26.0
3          0.0
4          0.0
          ... 
304526     0.0
304527     0.0
304528     0.0
304529     0.0
304530     0.0
Name: OWN_CAR_AGE, Length: 304531, dtype: float64

In [64]:
#处理EXT_SOURCE_1和EXT_SOURCE_3

mean_val = app_tr["EXT_SOURCE_1"].mean()
app_tr["EXT_SOURCE_1"].fillna(mean_val, inplace=True)
mean_val = app_tr["EXT_SOURCE_3"].mean()
app_tr["EXT_SOURCE_3"].fillna(mean_val, inplace=True)


In [65]:
t=missing_values_table(app_tr,"application_{train|test}.csv")
t

Total 6 columns missing values


,Row,Description,Special,Missing Values,% of Total Values,count,mean,std,min,25%,50%,75%,max
0,AMT_REQ_CREDIT_BUREAU_YEAR,Number of enquiries to Credit Bureau...,NaN,41108,13.5,263423.0,1.905904,1.869645,0.0,0.0,1.0,3.0,25.0
1,AMT_REQ_CREDIT_BUREAU_QRT,Number of enquiries to Credit Bureau...,NaN,41108,13.5,263423.0,0.266127,0.795735,0.0,0.0,0.0,0.0,261.0
2,AMT_REQ_CREDIT_BUREAU_MON,Number of enquiries to Credit Bureau...,NaN,41108,13.5,263423.0,0.267782,0.915330,0.0,0.0,0.0,0.0,27.0
3,AMT_REQ_CREDIT_BUREAU_WEEK,Number of enquiries to Credit Bureau...,NaN,41108,13.5,263423.0,0.034484,0.204615,0.0,0.0,0.0,0.0,8.0
4,AMT_REQ_CREDIT_BUREAU_DAY,Number of enquiries to Credit Bureau...,NaN,41108,13.5,263423.0,0.006981,0.110358,0.0,0.0,0.0,0.0,9.0
5,AMT_REQ_CREDIT_BUREAU_HOUR,Number of enquiries to Credit Bureau...,NaN,41108,13.5,263423.0,0.006385,0.083786,0.0,0.0,0.0,0.0,4.0


In [66]:
# 处理 AMT_REQ_CREDIT_BUREAU

for s in t["Row"].tolist():
    app_tr[s].fillna(0, inplace=True)

In [67]:
t=missing_values_table(app_tr,"application_{train|test}.csv")
t

Total 0 columns missing values


,Row,Description,Special,Missing Values,% of Total Values,count,mean,std,min,25%,50%,75%,max


至此，主表缺失值处理完毕。
此外，也对app_te表进行处理


In [68]:
app_te = pd.read_csv(app_te_path)

t = missing_values_table(app_te, "application_{train|test}.csv")
t_small=t.loc[t["% of Total Values"]<3]
t_large=t.loc[t["% of Total Values"]>3]
app_te.dropna(subset=t_small["Row"],
          axis=0, # axis=0表示删除行；
          how='any', # how=any表示若列name、age中，任意一个出现空值，就删掉该行
          inplace=True # inplace=True表示在原df上进行修改；
          )
app_te = app_te.reset_index(drop=True)

t = missing_values_table(app_te, "application_{train|test}.csv")
temp=t.drop(columns=["Special"]).isnull().T.any()
temp.loc[temp==True].index
rows=t.loc[temp.loc[temp==True].index].Row
for col in rows: 
    app_te[str(col)] = app_te[str(col)].fillna(value="MyNull")

t = missing_values_table(app_te, "application_{train|test}.csv")
t_house=t.loc[t["Description"]=="Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor"]
temp=app_te.loc[:,t_house["Row"].tolist()].isnull().sum(axis=1)
app_te["MY_HOUSE_OWN"]=temp
app_te_drop_list=t_house["Row"].tolist()
for col in app_te_drop_list: 
    app_te[col] = app_te[col].fillna(value=0)

t=missing_values_table(app_te,"application_{train|test}.csv")
t

Total 64 columns missing values
Total 57 columns missing values
Total 52 columns missing values
Total 9 columns missing values


,Row,Description,Special,Missing Values,% of Total Values,count,mean,std,min,25%,50%,75%,max
0,OWN_CAR_AGE,Age of client's car,NaN,31675,66.3,16097.0,11.791762,11.450478,0.000000,4.000000,9.000000,15.000000,74.000000
1,EXT_SOURCE_1,Normalized score from external data ...,normalized,20161,42.2,27611.0,0.500959,0.205320,0.013458,0.343060,0.506374,0.665975,0.939145
2,EXT_SOURCE_3,Normalized score from external data ...,normalized,8502,17.8,39270.0,0.499675,0.189602,0.000527,0.362277,0.519097,0.652897,0.882530
3,AMT_REQ_CREDIT_BUREAU_YEAR,Number of enquiries to Credit Bureau...,NaN,5951,12.5,41821.0,1.976256,1.839608,0.000000,0.000000,2.000000,3.000000,17.000000
4,AMT_REQ_CREDIT_BUREAU_QRT,Number of enquiries to Credit Bureau...,NaN,5951,12.5,41821.0,0.550274,0.694956,0.000000,0.000000,0.000000,1.000000,7.000000
5,AMT_REQ_CREDIT_BUREAU_MON,Number of enquiries to Credit Bureau...,NaN,5951,12.5,41821.0,0.009445,0.111859,0.000000,0.000000,0.000000,0.000000,6.000000
6,AMT_REQ_CREDIT_BUREAU_WEEK,Number of enquiries to Credit Bureau...,NaN,5951,12.5,41821.0,0.002822,0.054379,0.000000,0.000000,0.000000,0.000000,2.000000
7,AMT_REQ_CREDIT_BUREAU_DAY,Number of enquiries to Credit Bureau...,NaN,5951,12.5,41821.0,0.001841,0.046611,0.000000,0.000000,0.000000,0.000000,2.000000
8,AMT_REQ_CREDIT_BUREAU_HOUR,Number of enquiries to Credit Bureau...,NaN,5951,12.5,41821.0,0.002104,0.046343,0.000000,0.000000,0.000000,0.000000,2.000000


In [29]:
# 可以发现和train表是一致的

app_te["OWN_CAR_AGE"] = app_te["OWN_CAR_AGE"].fillna(value=0)
mean_val = app_te["EXT_SOURCE_1"].mean()
app_te["EXT_SOURCE_1"].fillna(mean_val, inplace=True)
mean_val = app_te["EXT_SOURCE_3"].mean()
app_te["EXT_SOURCE_3"].fillna(mean_val, inplace=True)
t=missing_values_table(app_te,"application_{train|test}.csv")
for s in t["Row"].tolist():
    app_te[s].fillna(0, inplace=True)
t=missing_values_table(app_te,"application_{train|test}.csv")
t

Total 6 columns missing values
Total 0 columns missing values


,Row,Description,Special,Missing Values,% of Total Values,count,mean,std,min,25%,50%,75%,max


In [70]:
# 最后把准备删除的行给删除了

app_tr.drop(columns=list(set(app_tr_drop_list+app_te_drop_list)),inplace=True)
app_te.drop(columns=list(set(app_tr_drop_list+app_te_drop_list)),inplace=True)
print(app_tr.shape)
print(app_te.shape)

(304531, 80)
(47772, 79)


In [73]:
# 结果保存

app_tr.to_csv(result_dir+"\\application_train.csv",index=False)
app_te.to_csv(result_dir+"\\application_test.csv",index=False)